# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов, в которой отражены данные о количестве детей, трудовом стаже, образовании, семейном статусе, а также о типе занятости, ежемесячном доходе и цели получения кредита.  
Выполнять наше исследование будем в следующем порядке:
1. Подробно изучим файл с данными
2. Обработаем пропуски и  сделаем проверку типов данных
3. Удалим дубликаты 
4. Категоризируем данные 
5. Проведем рассчеты для каждой категории
6. Сделаем конечный вывод

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропущенных значений](#null)
    * [Замена типов данных](#change)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemma)
    * [Категоризация](#category)
3. [Проверка гипотез](#check)
4. [Заключение](#final)

### <a id="start"> Шаг 1. Откройте файл с данными и изучите общую информацию. </a> 

In [1]:
#импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла  данных
clients = pd.read_csv('/datasets/data.csv') 
# получение общей информаци о таблице и вывод первых 10 строк
clients.info() 
clients.head(10) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод 

В таблице 12 столбцов, они имеют разные типы данных: object, int, float. Количсество значений в столбцах различается, следовательно, в данных есть пропущенные значения. Для исследования особенно ценны столбцы: children, family_status, debt, total_type. Частично не заполнены значения в колонках days_employed и total_income. Один из возможных вариантов объяснения, что данные клиенты не имеют трудового стажа и, следовательно, не имеют зарегистрированного ежемесячного дохода. Вызывают вопрос отрицательные значания в колонке days_employed. Возможно, не было заданого шаблона заполнения и минут с данном варианте тире. В колонке education занчения разного регистра.  Интерпритация названия столбцов clients таблицы:
* children — количество детей в семье  
* days_employed — общий трудовой стаж в днях  
* dob_years — возраст клиента в годах  
* education — уровень образования клиента  
* education_id — идентификатор уровня образования  
* family_status — семейное положение  
* family_status_id — идентификатор семейного положения  
* gender — пол клиента  
* income_type — тип занятости  
* debt — имел ли задолженность по возврату кредитов  
* total_income — ежемесячный доход  
* purpose — цель получения кредита  

### <a id="preprocessing"> Шаг 2. Предобработка данных </a>

### <a id="null"> Обработка пропусков </a> 

In [3]:
#вывляем количество пропусков
clients.isnull().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
#рассмотрим первые 5 строк, где пропущены данные в столбце дохода
clients[clients['total_income'].isnull()].head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [5]:
# предположение: трудовой стаж не так важен для исследования, в отличии от дохода.
#заменяем трудовой стаж на 0
clients['days_employed'] = clients['days_employed'].fillna(0) 
#расчет медианного дохода по типу занятости клиента и вывод на результата
#медиану используем, так как это значение менее чуствительно к выбросам, а в столбце доход это высоковероятно
median_income = clients.groupby('income_type')['total_income'].median().reset_index()
median_income

,income_type,total_income
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,150447.935283
3,компаньон,172357.950966
4,пенсионер,118514.486412
5,предприниматель,499163.144947
6,сотрудник,142594.396847
7,студент,98201.625314


In [6]:
#заменяем пропущенные значения в колоннке дохода на медианное значение для каждой категории:
#создаем цикл для уникальных значений из столбца Типа занятости, считаем медианное значение дохода для каждого типа и
#заменяем ими пропущенные значения
for row in clients['income_type'].unique():
    median_income = clients.loc[clients['income_type'] == row,'total_income'].median()
    clients.loc[clients['income_type'] == row, 'total_income'] = clients['total_income'].fillna(median_income)



In [7]:
#проверяем количество пропусков
clients.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

На этом этапе были отработаны пропуски. Пропущенные значения в стаже были заменены нулями, так как для нашего исследования это значение не является ключевым. Значения в столбце дохода были заменены на медианные в зависимости от типа занятости клиента. Медианные значения были применены, так как расчет по среднему более чувствителен к выбросам, а по доходу это высоковероятно.

### <a id="change"> Замена типа данных </a>

In [8]:
#Заменяем тип данных в столбце трудового стажа и дохода на целочисленный тип
try:
    clients['days_employed'] = clients['days_employed'].astype('int')
except:
    "Эти значения невозможно изменить"
clients['total_income'] = clients['total_income'].astype('int')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Для изменения типа дынных на целочесленный был применен метод astype, так как значение столбца трудового стажа и дохода были изначально вещественного типа. 

### <a id="duplicates"> Обработка дубликатов </a>

In [9]:
#приведем столбец образования к единому регистру
clients['education'] = clients['education'].str.lower()
#проверяем количество дубликатов
clients.duplicated().sum()

71

In [10]:
# удаляем дубликаты и делаем проверку
clients = clients.drop_duplicates().reset_index(drop = True)
clients.duplicated().sum()

0

In [11]:
# проверяем уникальные значение по столбцу Дети
clients['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [12]:
#Удалим значения "20" и "-1". Возможно нужно заменить -1 по модулю???
clients = clients.loc[~clients['children'].isin([20])]
clients = clients.loc[~clients['children'].isin([-1])]

clients['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [13]:
# смотрим уникальные значения по столбцу Семейный статус
clients['family_status'].value_counts() 

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

In [14]:
#смотрим уникальные значения в Целях кредита. Это потребуется для дальнейше группировки
clients['purpose'].value_counts() 

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      616
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

### Вывод

В колонке Дети были выявлены некорректные значения, которые могли быть полученны ошибкой ввода. Отрицательные значения и "20" были исключены их результатов, их количество не превышало 1% выдачи (123 значения).
По калонке Семейного статуса нет замечаний.
В колонке причин много дубликатов одинаковых по смыслу, их нужно группировать. Были выявлены цели: свадьба, недвижимость (жилье), автомобиль, образование
По всему датафрейму было найдено 71 дубликат, они были удалены.

### <a id="lemma"> Лемматизация </a>

In [15]:
#создадим функцию, чтобы получить новый столбец, в котором Цель кредита будет лемматизирована.
from pymystem3 import Mystem
m = Mystem()

def new_name_purpose(purpose):
    purpose_new = ' '.join(m.lemmatize(purpose))
    return purpose_new

clients['purpose_new_lem'] = clients['purpose'].apply(new_name_purpose)
clients.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new_lem
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n


### Вывод

На данном этапе был применен метод лемматизации к колонке целей. Каждая причина к колонке причины была разбита на отдельные слова и приведена к своей словарной форме. Как результат были определены основные группы целей, по которым дальше будем категоризировать: свадьба, недвижимость, жилье, автомобиль, образование.

### <a id="category"> Категоризация данных </a>

In [16]:
#исходные данные уже категоризированы по количеству детей
#выведем количество в каждой категории
children_count = clients.groupby('children')['debt'].count()
children_count

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: debt, dtype: int64

In [17]:
#рассмотрим минимальные, максимальные и медианные значения дохода по типу занятоси.
#на осове этих данных дудем делать классификацию
clients.groupby('income_type').agg({'total_income': ['min','max','median','count']})


total_income                        
                         min      max  median  count
income_type                                         
безработный            59956   202722  131339      2
в декрете              53829    53829   53829      1
госслужащий            29200   910451  150447   1451
компаньон              28702  2265604  172357   5047
пенсионер              20667   735103  118514   3812
предприниматель       499163   499163  499163      2
сотрудник              21367  1726276  142594  11015
студент                98201    98201   98201      1

In [18]:
# воспользуемся методом qcut для того чтобы сделать выборку по доходу таким образом, чтобы в каждом интервале было
# Оинаковое количество записей
quintiles = pd.qcut(clients['total_income'], 5)
pd.value_counts(quintiles)

(20666.999, 98514.0]     4267
(214604.0, 2265604.0]    4266
(161380.0, 214604.0]     4266
(132113.0, 161380.0]     4266
(98514.0, 132113.0]      4266
Name: total_income, dtype: int64

In [19]:
#приступаем к категоризации клиентов по доходу
def income_group(total_income):
    if total_income <= 98514:
        return 'минимальный'
    if total_income <= 132113:
        return 'низкий'
    if total_income <= 161380:
        return 'средний'
    if total_income <= 214604:
        return 'выше среднего'
    return 'высокий'
#добавляем столбец категории клиентов по уровню дохода
clients['income_group'] = clients['total_income'].apply(income_group)
# выводим первые 5 сток для проверки
clients.head(5)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new_lem,income_group
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,высокий
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,низкий
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,средний
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,средний


In [20]:
#вводим классификацию по целям кредита
def purpose_group(purpose_new_lem):
    if 'недвижимость' in purpose_new_lem or 'жилье' in purpose_new_lem:
        return 'недвижимость'
    if 'свадьба' in purpose_new_lem:
        return 'свадьба'
    if 'автомобиль' in purpose_new_lem:
        return 'автомобиль'
    if 'образование' in purpose_new_lem:
        return 'образование'
    
#добавляем столбец категории клиентов по уровню дохода
clients['purpose_group'] = clients['purpose_new_lem'].apply(purpose_group)
# выводим первые 10 сток для проверки
clients.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new_lem,income_group,purpose_group
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,высокий,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,низкий,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,средний,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,высокий,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,средний,свадьба


In [21]:
# создадим словарь для категоризации клиентов по семейному статусу
# для начала выведем уникальные значения
clients['family_status'].value_counts() 

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

In [22]:
#создаем словарь
status_dict = {'married': 'женат / замужем', 'common_law': 'гражданский брак', 'not_married': 'Не женат / не замужем', 'widow': 'вдовец / вдова' }
#посмотрим наличие задолжености по самому распространенному семейному статусу
clients.loc[clients['family_status']== status_dict['married'],'debt'].sum()

927

### Вывод

На данном этапе была добавлена классификация клиентов по количеству детей, уровню дохода и целям кредита. Категоризация по количеству детей была дана уже в начальных данных. По уровню дохода разделили клиентов на доход минимальный, низкий, средний, выше среднего и высокий. Чтобы разделить всю выборку на диапазоны с одинаковым количеством входящих значений, использовали метод qcut. По целям кредита ввели следующую классификацию: недвижимость, свадьба, автомобиль, образование. Данный выбор наименований был обусловлен тем, что на этапе обработки дубликатов нами были выведены все уникальные значения из столбца "Цели кредита". После изучения этих причин логически были определены именно эти категории.

### <a id="check"> Шаг 3. Проверка гипотез </a> 

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [23]:
#Зависимость между наличием детей и возвратом кредита в срок
#делаем расчет кредитов с просрочкой по категориям
#найдем общее количество клиентов по категориям
#получим долю просроченых кредитов по категориям

children_debt = clients.groupby('children').agg({'debt': ['sum','count','mean']})
children_debt.sort_values(by = ('debt', 'mean')).round(decimals = 3)

debt              
           sum  count   mean
children                    
5            0      9  0.000
0         1063  14091  0.075
3           27    330  0.082
1          444   4808  0.092
2          194   2052  0.095
4            4     41  0.098

### Вывод

Можно отметить, что клиенты без детей более дисциплинированы при выплате кредита. Из наших данных следует, что клиенты с 5 детьми не допускают опаздания по срокам выплаты. Но в нашей выборке таких клиентов всего 9, поэтому данных не достаточно, чтобы сказать, что эта категория клиентов самая надежная.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
#зависимость между семейным положением и возвратом кредита в срок
#делаем расчет кредитов с просрочкой по категориям
#найдем общее количество клиентов по категориям
#получим долю просроченых кредитов по категориям
family_status_debt = clients.groupby('family_status').agg({'debt': ['sum','count','mean']})
family_status_debt.sort_values(by = ('debt', 'mean')).round(decimals = 3)

debt              
                       sum  count   mean
family_status                           
вдовец / вдова          63    951  0.066
в разводе               84   1189  0.071
женат / замужем        927  12261  0.076
гражданский брак       385   4134  0.093
Не женат / не замужем  273   2796  0.098

### Вывод

Вдовы и вдовцы самые дисциплинированные плательщики. Не женатые чаще других пропускают оплату

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
#зависимость между уровнем дохода и возвратом кредита в срок
#делаем расчет кредитов с просрочкой по категориям
#найдем общее количество клиентов по категориям
#получим долю просроченых кредитов по категориям

income_group_debt = clients.groupby('income_group').agg({'debt': ['sum','count','mean']})
income_group_debt.sort_values(by = ('debt', 'mean')).round(decimals = 3)

debt             
               sum count   mean
income_group                   
высокий        299  4266  0.070
минимальный    344  4267  0.081
выше среднего  358  4266  0.084
низкий         358  4266  0.084
средний        373  4266  0.087

### Вывод

При применении категоризации методом qcut (на равные доли выборки) можно сказать, что клиенты с высоким доходом реже других не выплачиват кредит. Платежная дисциплина у остальных категорий клиентов находится на примерно одинаковом уровне.

- Как разные цели кредита влияют на его возврат в срок?

In [26]:
#влияние цели кредита на его возврат в срок
#найдем общее количество клиентов по категориям
#получим долю просроченых кредитов по категориям

purpose_group_debt = clients.groupby('purpose_group').agg({'debt': ['sum','count','mean']})
purpose_group_debt.sort_values(by = ('debt', 'mean')).round(decimals = 3)

debt              
               sum  count   mean
purpose_group                   
недвижимость   780  10751  0.073
свадьба        183   2313  0.079
образование    369   3988  0.093
автомобиль     400   4279  0.093

### Вывод

Лучше всего платят по кредитам на недвижимость. Можно это объяснить тем, что при пропуске оплаты, например, по ипотеке, процент кредита значительно увеличивается, а объем кредита и срок остается довольно большим.

In [27]:
#сделаем сфодную таблицу, чтобы определить какие цели кредитов наибелее популярны по каждой категории семейного статуса
clients_pivot = clients.pivot_table(index = 'purpose_group', columns = 'family_status', values = 'debt', aggfunc = 'count')
clients_pivot

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
purpose_group,,,,,
автомобиль,635.0,279.0,218.0,430.0,2717.0
недвижимость,1586.0,672.0,535.0,988.0,6970.0
образование,575.0,238.0,198.0,403.0,2574.0
свадьба,NaN,NaN,NaN,2313.0,NaN


### Вывод

Недвижимость - самая популярная цель кредита почти для каждой категории семейного статуса!

### <a id="final"> Шаг 4. Общий вывод </a>

В ходе исследования на основе статистики платежеспособности клиентов было проверено, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Было подтверждено, что в каждой категории клиентов в зависимости от того или иного признака, клиенты платят по-разному.  Не женатые чаще других пропускают оплату, вдовцы и разведенные самые дисциплинированные плательщики.  Наблюдается прямая закономерность допущения пропуска в выплате кредида в зависимости от количества детей.  Клиенты с высоким доходом реже других не выплачиват кредит. Платежная дисциплина у остальных категорий клиентов находится на примерно одинаковом уровне.  Лучше всего платят по кредитам на недвижимость. Кредит на образование и автомобили чаще остальных не погашаются в срок.